In [ ]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lit
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id

conf = SparkConf() 
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
dfTwitter = spark.read.option("multiline","true").json('s3a://ninastack-sptech-bucket-bruto/tweets.json')

In [ ]:
from pyspark.sql.functions import col

df = dfTwitter.withColumn('id', col('tweet.id')).withColumn('data', col('tweet.data')).withColumn('nome', col('tweet.nome')).withColumn('texto', col('tweet.texto'))


In [ ]:
_dfTweets = df.select(
    col('texto').alias('OPINIÃO')
)
nubank = _dfTweets.filter("texto like '%nubank%' OR texto like '%#nubank%'").withColumn("BANCO",lit("NUBANK"))


In [ ]:
inter = _dfTweets.filter("texto like '%inter%' OR texto like '%#inter%'").withColumn("BANCO",lit("INTER"))

In [ ]:
safra = _dfTweets.filter("texto like '%safra%' OR texto like '%#safra%'").withColumn("BANCO",lit("SAFRA"))

In [ ]:
santander = _dfTweets.filter("texto like '%santander%' OR texto like '%#santander%'").withColumn("BANCO",lit("SANTANDER"))

In [ ]:
c6 = _dfTweets.filter("texto like '%c6%' OR texto like '%#c6%'").withColumn("BANCO",lit("C6BANK"))

In [ ]:
dfOpinioes = nubank.union(inter).union(safra).union(santander).union(c6)

In [ ]:
dfOpinioes.show()

In [ ]:
dfOpinioes.write.option("header", True).csv("s3a://ninastack-sptech-bucket-cliente/opinioes_twitter", emptyValue='') 

In [ ]:
dfTweetsSingleCol = _dfTweets.select(concat(*_dfTweets.columns).alias('data'))

In [ ]:
dfTweetsSingleCol.write.mode('append').text('s3a://ninastack-sptech-bucket-tratados/opinioes_twitter.txt')

In [ ]:
dfCsv = spark.read.option('delimiter', ';').option('header', 'true').csv('s3a://ninastack-sptech-bucket-bruto/transacoes_pix.csv')

In [ ]:
dfCsv.show(vertical=True)

In [ ]:
_dfCsv = dfCsv.select(
    col('valor').alias('VALOR TRANSFERIDO (R$)'),
    col('tipoChave').alias('TIPO CHAVE'),
    col('nomeInstituicao').alias('NOME INSTITUIÇÃO'),
    col('siglaEstado').alias('UF')
)
# _dfCsv.show()



In [ ]:
dfCsvSingleCol = _dfCsv.select(concat(*_dfCsv.columns).alias('data'))
# dfCsvSingleCol.show()

In [ ]:
dfCsvSingleCol.write.mode('overwrite').text('s3a://ninastack-sptech-bucket-tratados/transacoes_pix.txt') 

In [ ]:
dfTxt = spark.read.option('delimiter', ';').option('header', 'true').csv('s3a://ninastack-sptech-bucket-bruto/pessoa.txt')

In [ ]:
dfTxt.show()

In [ ]:
_dfTxt = dfTxt.select(
    col('idade').cast(IntegerType()).alias('IDADE'),
)
#_dfTxt.show()

In [ ]:
dfTxtSingleCol = _dfTxt.select(concat(*_dfTxt.columns).alias('data'))
# dfTxtSingleCol.show()

In [ ]:
dfTxtSingleCol.write.mode('overwrite').text('s3a://ninastack-sptech-bucket-tratados/pessoa.txt')

In [ ]:
dfJson = spark.read.json('s3a://ninastack-sptech-bucket-bruto/fraudes_pix.json')

In [ ]:
dfJson.show(vertical = True)

In [ ]:
_dfJson = dfJson.select(
    col('valor').alias('VALOR FRAUDES')
)
#_dfJson.show()

In [ ]:
dfJsonSingleCol = _dfJson.select(concat(*_dfJson.columns).alias('data'))
# dfJsonSingleCol.show()

In [ ]:
dfJsonSingleCol.write.mode('overwrite').text('s3a://ninastack-sptech-bucket-tratados/fraudes_pix.txt')

In [ ]:
import pandas as pd
pdfXml = pd.read_xml('s3a://ninastack-sptech-bucket-bruto/DICT.xml')

In [ ]:
dfXml = spark.createDataFrame(pdfXml)


In [ ]:
_dfXml = dfXml.select(
    col('tipoConta').alias('TIPO CONTA'),
    col('reivindicadaDoacao').alias('CHAVE REIVINDICADA POR DOAÇÃO'),
)
# _dfXml.show()

In [ ]:
dfXmlSingleCol = _dfXml.select(concat(*_dfXml.columns).alias('data'))


In [ ]:
dfXmlSingleCol.write.mode('overwrite').text('s3a://ninastack-sptech-bucket-tratados/DICT.txt')

In [ ]:
_dfCsv = _dfCsv.withColumn("id", monotonically_increasing_id())

In [ ]:
_dfTxt = _dfTxt.withColumn("id", monotonically_increasing_id())

In [ ]:
_dfJson = _dfJson.withColumn("id", monotonically_increasing_id())

In [ ]:
_dfXml = _dfXml.withColumn("id", monotonically_increasing_id())

In [ ]:
df_seguranca = _dfCsv.join(_dfTxt, on="id", how="outer").join(_dfJson, on="id", how="outer").join(_dfXml, on="id", how="outer") 

In [ ]:
# df_sample = df_segurancaMaster.sample(0.001, seed=1234)

In [ ]:
df_seguranca.write.option("header", True).csv("s3a://ninastack-sptech-bucket-cliente/security", emptyValue='') 

In [ ]:
df_seguranca.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://server-gp-nina.database.windows.net;databaseName=database-gp06-nina;') \
 .option('dbtable', 'dbo.Security') \
 .option('user', 'adm') \
 .option('password', '@Urubu100') \
 .save()

In [ ]:
dfOpinioes.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://server-gp-nina.database.windows.net;databaseName=database-gp06-nina;') \
 .option('dbtable', 'dbo.OpinioesTwitter') \
 .option('user', 'adm') \
 .option('password', '@Urubu100') \
 .save()

In [1]:
import socket
print(socket.gethostbyname(socket.gethostname()))


10.100.0.245
